In [1]:
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.memory import ConversationBufferMemory
import openai
import tempfile

# 스트림릿 페이지 설정
st.set_page_config(page_title="RAG with Streamlit", page_icon=":books:")

# 사이드바: OpenAI API 키 입력
st.sidebar.title("설정")
openai_api_key = st.sidebar.text_input("OpenAI API Key를 입력하세요", type="password")
if openai_api_key:
    openai.api_key = openai_api_key

# 사이드바: 깃허브 링크
st.sidebar.markdown("### Github Repo")
st.sidebar.markdown("[프로젝트 깃허브 링크](https://github.com/your-repo-url)")

# 페이지 헤더
st.title("RAG 파이프라인 데모 - 1984 소설 예시")

# 세션 스테이트 초기화
if "memory" not in st.session_state:
    st.session_state["memory"] = ConversationBufferMemory(
        memory_key="chat_history", input_key="question",
    )
if "vectorstore" not in st.session_state:
    st.session_state["vectorstore"] = None
if "uploaded_file" not in st.session_state:
    st.session_state["uploaded_file"] = None

# 파일 업로드
uploaded_file = st.file_uploader(
    "문서 파일을 업로드하세요 (예: 1984 일부 텍스트)", type=["txt"]
)
if uploaded_file is not None:
    st.session_state["uploaded_file"] = uploaded_file


# 벡터 스토어 생성
def create_vectorstore(file):
    # 임시 파일로 저장
    with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as tmp_file:
        tmp_file.write(file.read())
        tmp_file_path = tmp_file.name

    loader = UnstructuredFileLoader(tmp_file_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(splits, embeddings)
    return vectorstore


# 문서를 기반으로 RAG 체인 구성 함수
def build_rag_chain():
    # 프롬프트 템플릿
    prompt_template = """
당신은 George Orwell의 1984 소설에 관한 질문에 답변하는 도우미입니다.
주어진 문맥 정보를 기반으로만 답변하고, 문맥 정보에서 발견할 수 없는 내용은 "문맥 정보에서 찾을 수 없습니다"라고 대답하세요.

문맥 정보:
{context}

이전 대화:
{chat_history}

질문: {question}
답변:
"""
    document_prompt = PromptTemplate(
        input_variables=["page_content"], template="{page_content}"
    )
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question", "chat_history"],
    )

    # LLM 설정 (모델명은 필요에 따라 수정하세요)
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    # LLMChain
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
    )
    return stuff_chain


def ask_question(question):
    if st.session_state["vectorstore"] is None:
        return "먼저 문서를 업로드해 주세요."

    retriever = st.session_state["vectorstore"].as_retriever()
    docs = retriever.get_relevant_documents(question)

    # 메모리에서 채팅 기록 가져오기
    chat_history = st.session_state["memory"].load_memory_variables({})["chat_history"]

    # StuffDocumentsChain 실행
    stuff_chain = build_rag_chain()
    response = stuff_chain.run(
        input_documents=docs, question=question, chat_history=chat_history
    )

    # 메모리 업데이트
    st.session_state["memory"].save_context(
        {"question": question}, {"output": response}
    )

    return response


# 파일 업로드 후 벡터 스토어 생성
if st.session_state["uploaded_file"] is not None and openai_api_key:
    if st.session_state["vectorstore"] is None:
        st.write("문서를 벡터 스토어로 변환 중입니다...")
        st.session_state["vectorstore"] = create_vectorstore(
            st.session_state["uploaded_file"]
        )
        st.success("벡터 스토어 생성 완료!")

# 사용자 질문 입력
question = st.text_input("질문을 입력하세요:", "")
if st.button("질문하기"):
    if not openai_api_key:
        st.warning("OpenAI API Key를 입력해야 합니다.")
    elif st.session_state["uploaded_file"] is None:
        st.warning("문서 파일을 먼저 업로드해주세요.")
    else:
        with st.spinner("답변 생성 중..."):
            answer = ask_question(question)
        st.write("**Q:**", question)
        st.write("**A:**", answer)

# 이전 대화 히스토리 출력
if st.session_state["memory"].buffer:
    st.markdown("---")
    st.markdown("### 대화 기록")

    for i, msg in enumerate(st.session_state["memory"].buffer):
        role = "사용자" if i % 2 == 0 else "AI"
        # 각 메시지를 바로 출력
        st.write(f"**{role}**: {msg}")


2025-03-25 14:52:44.446 
  command:

    streamlit run /Users/hwang-isaac/.pyenv/versions/fullgpt/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-25 14:52:44.446 Session state does not function when running a script without `streamlit run`


KeyError: 'st.session_state has no key "uploaded_file". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization'